# Summary

* This is a tutorial on using R (& Python via [Reticulate](https://rstudio.github.io/reticulate/)) 
  for accessing the scBaseCamp dataset hosted by the Arc Institute.
* The data can be streamed or downloaded locally.
  * For small jobs (e.g., summarizing the some metadata), streaming is recommended.
  * For large jobs (e.g., training a model), downloading is recommended.
* See the [README](README.md#metadata) for a description of the obs metadata.

## IMPORTANT NOTICE

> We are converting file extensions from `parquet.gz` and `h5ad.gz` to `parquet` and `h5ad`, respectively. The `*.gz` files will be **deleted on Friday, Feb 26 2025 at 5:00 PM PST.**
You can copy just the `parquet` and `h5ad`, files via: `rsync`:

```bash
gsutil -m rsync -r -x "^(?\!.*\.parquet$)" "gs://arc-ctc-scbasecamp/2025-02-25/metadata"
```

```bash
gsutil -m rsync -r -x "^(?\!.*\.h5ad$)" "gs://arc-ctc-scbasecamp/2025-02-25/h5ad"
```

**Sorry for the inconvience!**


# Setup

### Installation

If needed, install the necessary dependencies.

You can use the [conda environment](../conda_envs/R.yml) provided in this git repository.

# Load libraries

In [1]:
# Load required libraries
library(dplyr)
library(arrow)
library(reticulate)
library(googleCloudStorageR)
os = import("os")
pd = import("pandas")
ad = import("anndata")
gcsfs = import("gcsfs")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘arrow’


The following object is masked from ‘package:utils’:

    timestamp




In [2]:
# set max print rows
options(repr.matrix.max.rows=4)

# Data location

In [3]:
# GCS bucket path
gcs_base_path = "gs://arc-ctc-scbasecamp/2025-02-25"

In [4]:
# which STARsolo feature type to use?
feature_type = "GeneFull_Ex50pAS"

# List available files

Let's see what we have to work with!

In [5]:
# initialize GCS file system for reading data from GCS
fs = gcsfs$GCSFileSystem(
    token = Sys.getenv("GOOGLE_APPLICATION_CREDENTIALS")
)

In [6]:
# helper function
get_parquet_files = function(gcs_base_path, feature_type, target = NULL, endswith = NULL) {
  files = fs$glob(os$path$join(gcs_base_path, feature_type, "**"))
  

  if (!is.null(target)) {
    files = files[sapply(files, function(f) basename(f) == target)]
  } else if (!is.null(endswith)) {
    files = files[sapply(files, function(f) grepl(paste0(endswith, "$"), f))]
  }
  
  file_list = lapply(files, function(f) {
    parts = unlist(strsplit(f, "/"))
    c(parts[length(parts)-1], f)
  })
  
  file_df = as.data.frame(do.call(rbind, file_list), stringsAsFactors = FALSE)
  colnames(file_df) = c("organism", "file_path")
  
  return(file_df)
}

In [7]:
# function to read parquet files from GCS
read_data = function(file, n = NULL){
  if(!startsWith(file, "gs://")){
    file = paste0("gs://", file)
  }
  dataset = open_dataset(file, format = "parquet")
  if (is.null(n)) {
    dataset %>% collect() %>% as.data.frame()
  } else {
    dataset %>% head(n) %>% collect() %>% as.data.frame()
  }
}

## Parquet files

* Contain the obs metadata
* Similar to CSV files, but more efficient for large datasets

In [8]:
# set the path to the metadata files
gcs_path = file.path(gcs_base_path, "metadata")
gcs_path

[1] "gs://arc-ctc-scbasecamp/2025-02-25/metadata"

In [9]:
# list files
sample_pq_files = get_parquet_files(gcs_path, feature_type, "sample_metadata.parquet.gz")
sample_pq_files 

organism,file_path
<chr>,<chr>
Arabidopsis_thaliana,arc-ctc-scbasecamp/2025-02-25/metadata/GeneFull_Ex50pAS/Arabidopsis_thaliana/sample_metadata.parquet.gz
Bos_taurus,arc-ctc-scbasecamp/2025-02-25/metadata/GeneFull_Ex50pAS/Bos_taurus/sample_metadata.parquet.gz
⋮,⋮
Sus_scrofa,arc-ctc-scbasecamp/2025-02-25/metadata/GeneFull_Ex50pAS/Sus_scrofa/sample_metadata.parquet.gz
Zea_mays,arc-ctc-scbasecamp/2025-02-25/metadata/GeneFull_Ex50pAS/Zea_mays/sample_metadata.parquet.gz


### Per-obs metadata

In [10]:
# list files
obs_pq_files = get_parquet_files(gcs_path, feature_type, "obs_metadata.parquet.gz")
obs_pq_files 

organism,file_path
<chr>,<chr>
Arabidopsis_thaliana,arc-ctc-scbasecamp/2025-02-25/metadata/GeneFull_Ex50pAS/Arabidopsis_thaliana/obs_metadata.parquet.gz
Bos_taurus,arc-ctc-scbasecamp/2025-02-25/metadata/GeneFull_Ex50pAS/Bos_taurus/obs_metadata.parquet.gz
⋮,⋮
Sus_scrofa,arc-ctc-scbasecamp/2025-02-25/metadata/GeneFull_Ex50pAS/Sus_scrofa/obs_metadata.parquet.gz
Zea_mays,arc-ctc-scbasecamp/2025-02-25/metadata/GeneFull_Ex50pAS/Zea_mays/obs_metadata.parquet.gz


# h5ad files

In [11]:
gcs_path = file.path(gcs_base_path, "h5ad")
gcs_path

[1] "gs://arc-ctc-scbasecamp/2025-02-25/h5ad"

In [12]:
# list files
h5ad_files = get_parquet_files(gcs_path, feature_type, endswith=".h5ad.gz")
h5ad_files 

organism,file_path
<chr>,<chr>
Arabidopsis_thaliana,arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex50pAS/Arabidopsis_thaliana/SRX15202187.h5ad.gz
Arabidopsis_thaliana,arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex50pAS/Arabidopsis_thaliana/SRX15202188.h5ad.gz
⋮,⋮
Zea_mays,arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex50pAS/Zea_mays/SRX8487984.h5ad.gz
Zea_mays,arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex50pAS/Zea_mays/SRX8781690.h5ad.gz


# Expore the observation metadata

* `obs` ≃ cell

### Per-sample

* Useful for quickly summarizing the per-sample metadata (a small file versus the entire obs metadata file; see below).
* For this tutorial, we will just read in a subset

In [13]:
# read the metadata files
num_organisms = 2
num_samples_per_organism = 10
sample_metadata = sample_pq_files %>%
  pull(file_path) %>%
  head(n = num_organisms) %>%
  lapply(read_data, n = num_samples_per_organism) %>%
  bind_rows()
sample_metadata

entrez_id,srx_accession,file_path,obs_count,lib_prep,tech_10x,cell_prep,organism,tissue,disease,purturbation,cell_line,czi_collection_id,czi_collection_name
<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>
24123125,SRX17302366,gs://arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex50pAS/Arabidopsis_thaliana/SRX17302366.h5ad.gz,9036,10x_Genomics,3_prime_gex,single_cell,Arabidopsis thaliana,other,not specified,"BL (Brassinolide), 100nM, 0.5 hours post-treatment",WT Col-0,NA,NA
24123140,SRX17302381,gs://arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex50pAS/Arabidopsis_thaliana/SRX17302381.h5ad.gz,14317,10x_Genomics,3_prime_gex,single_cell,Arabidopsis thaliana,other,not specified,"control treatment, age: 7 days",WT Col-0,NA,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
26767424,SRX19487478,gs://arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex50pAS/Bos_taurus/SRX19487478.h5ad.gz,11477,10x_Genomics,3_prime_gex,single_cell,Bos taurus,blood,not specified,2 h LPS (Lipopolysaccharide),Peripheral blood mononuclear cells (PBMCs),NA,NA
27384248,SRX19992927,gs://arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex50pAS/Bos_taurus/SRX19992927.h5ad.gz,5851,10x_Genomics,3_prime_gex,single_cell,Bos taurus,milk,Not specified,Not specified,MiDC (Milk-derived adherent cells),NA,NA


In [14]:
# samples per organism
sample_metadata %>% count(organism)

organism,n
<chr>,<int>
Arabidopsis thaliana,10
Bos taurus,10


In [15]:
# cell counts per sample
sample_metadata %>% pull(obs_count) %>% summary()

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   3496    5720    8469    9413   11420   21404 

In [16]:
# cell count distribution per organism
sample_metadata %>%
  group_by(organism) %>%
  summarise(
    min = min(obs_count),
    max = max(obs_count),
    mean = mean(obs_count),
    median = median(obs_count),
    sd = sd(obs_count)
  )

organism,min,max,mean,median,sd
<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>
Arabidopsis thaliana,3496,20075,8688.0,7621.0,5306.759
Bos taurus,4120,21404,10138.8,8838.5,4923.111


### Per-obs metadata

In [17]:
# read the metadata files
num_organisms = 2
num_obs_per_organism = 1000
obs_metadata = obs_pq_files %>%
  pull(file_path) %>%
  head(n = num_organisms) %>%
  lapply(read_data, n = num_obs_per_organism) %>%
  bind_rows()
obs_metadata

gene_count,umi_count,SRX_accession,cell_barcode,__index_level_0__
<int>,<dbl>,<chr>,<chr>,<int>
3504,12028,SRX15202187,AAACCCAAGACCATGG,0
1610,3183,SRX15202187,AAACCCACAGGTCTCG,1
⋮,⋮,⋮,⋮,⋮
3639,9688,ERX13041271,AGAGCAGAGCTCCACG,998
3333,8109,ERX13041271,AGAGCAGAGGATTTAG,999


In [18]:
# gene count distribution
obs_metadata %>% pull(gene_count) %>% summary()

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    108    1850    3515    3407    4700   11929 

In [19]:
# umi counts distribution
obs_metadata %>% pull(umi_count) %>% summary()

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    504    3409    9170    9997   14204   64469 

#### Obs for particular samples

Let's get some obs metadata for particular samples

In [20]:
# re-print our selected samples
sample_metadata

entrez_id,srx_accession,file_path,obs_count,lib_prep,tech_10x,cell_prep,organism,tissue,disease,purturbation,cell_line,czi_collection_id,czi_collection_name
<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>
24123125,SRX17302366,gs://arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex50pAS/Arabidopsis_thaliana/SRX17302366.h5ad.gz,9036,10x_Genomics,3_prime_gex,single_cell,Arabidopsis thaliana,other,not specified,"BL (Brassinolide), 100nM, 0.5 hours post-treatment",WT Col-0,NA,NA
24123140,SRX17302381,gs://arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex50pAS/Arabidopsis_thaliana/SRX17302381.h5ad.gz,14317,10x_Genomics,3_prime_gex,single_cell,Arabidopsis thaliana,other,not specified,"control treatment, age: 7 days",WT Col-0,NA,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
26767424,SRX19487478,gs://arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex50pAS/Bos_taurus/SRX19487478.h5ad.gz,11477,10x_Genomics,3_prime_gex,single_cell,Bos taurus,blood,not specified,2 h LPS (Lipopolysaccharide),Peripheral blood mononuclear cells (PBMCs),NA,NA
27384248,SRX19992927,gs://arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex50pAS/Bos_taurus/SRX19992927.h5ad.gz,5851,10x_Genomics,3_prime_gex,single_cell,Bos taurus,milk,Not specified,Not specified,MiDC (Milk-derived adherent cells),NA,NA


In [21]:
# read the metadata files
target_organisms = sample_metadata %>% pull(organism) %>% unique() %>% gsub(" ", "_", .)
obs_metadata = obs_pq_files %>%
  filter(organism %in% target_organisms) %>%
  pull(file_path) %>%
  lapply(read_data) %>%
  bind_rows()

# join on SRX accession
obs_metadata = sample_metadata %>%
  inner_join(obs_metadata, by = c("srx_accession" = "SRX_accession"))
obs_metadata

entrez_id,srx_accession,file_path,obs_count,lib_prep,tech_10x,cell_prep,organism,tissue,disease,purturbation,cell_line,czi_collection_id,czi_collection_name,gene_count,umi_count,cell_barcode,__index_level_0__
<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<int>,<dbl>,<chr>,<int>
24123125,SRX17302366,gs://arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex50pAS/Arabidopsis_thaliana/SRX17302366.h5ad.gz,9036,10x_Genomics,3_prime_gex,single_cell,Arabidopsis thaliana,other,not specified,"BL (Brassinolide), 100nM, 0.5 hours post-treatment",WT Col-0,NA,NA,9126,77277,AAACCCAAGAGTGGCT,0
24123125,SRX17302366,gs://arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex50pAS/Arabidopsis_thaliana/SRX17302366.h5ad.gz,9036,10x_Genomics,3_prime_gex,single_cell,Arabidopsis thaliana,other,not specified,"BL (Brassinolide), 100nM, 0.5 hours post-treatment",WT Col-0,NA,NA,4300,11263,AAACCCAAGGCGATAC,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
27384248,SRX19992927,gs://arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex50pAS/Bos_taurus/SRX19992927.h5ad.gz,5851,10x_Genomics,3_prime_gex,single_cell,Bos taurus,milk,Not specified,Not specified,MiDC (Milk-derived adherent cells),NA,NA,464,766,TTTGTTGGTACAAAGT,5849
27384248,SRX19992927,gs://arc-ctc-scbasecamp/2025-02-25/h5ad/GeneFull_Ex50pAS/Bos_taurus/SRX19992927.h5ad.gz,5851,10x_Genomics,3_prime_gex,single_cell,Bos taurus,milk,Not specified,Not specified,MiDC (Milk-derived adherent cells),NA,NA,422,731,TTTGTTGGTGGGTCAA,5850


In [22]:
# summarize gene counts per organism
obs_metadata %>%
  group_by(organism) %>%
  summarise(
    min = min(gene_count),
    max = max(gene_count),
    mean = mean(gene_count),
    median = median(gene_count),
    sd = sd(gene_count)
  )

organism,min,max,mean,median,sd
<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>
Arabidopsis thaliana,119,14336,1959.175,1464,1605.399
Bos taurus,61,11248,2665.464,2317,1885.132


In [23]:
# summarize umi counts per organism
obs_metadata %>%
  group_by(organism) %>%
  summarise(
    min = min(umi_count),
    max = max(umi_count),
    mean = mean(umi_count),
    median = median(umi_count),
    sd = sd(umi_count)
  )

organism,min,max,mean,median,sd
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Arabidopsis thaliana,399,487844,7441.814,2583,16833.84
Bos taurus,500,321408,11217.109,6277,14692.41


# Load h5ad files

In [24]:
gcs_auth(json_file = Sys.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

In [25]:
# example
infile = "gs://arc-ctc-scbasecamp/2025-02-25/h5ad/Gene/Arabidopsis_thaliana/SRX16110574.h5ad.gz"

# split bucket and path
bucket = unlist(strsplit(infile, "/"))[3]
path = paste(unlist(strsplit(infile, "/"))[-(1:3)], collapse = "/")

In [26]:
# create a temp file
temp_file = tempfile(fileext = ".h5ad.gz")

In [27]:
# Download the file to a temporary location
gcs_get_object(object_name = path, bucket = bucket, saveToDisk = temp_file, overwrite = TRUE)

ℹ Downloading 2025-02-25/h5ad/Gene/Arabidopsis_thaliana/SRX16110574.h5ad.gz



✔ Saved 2025-02-25/h5ad/Gene/Arabidopsis_thaliana/SRX16110574.h5ad.gz to /tmp/R…





[1] TRUE

In [28]:
# read in anndata object
adata = ad$read_h5ad(temp_file)
adata

AnnData object with n_obs × n_vars = 5906 × 31109
    obs: 'gene_count', 'umi_count', 'SRX_accession'
    var: 'gene_symbols', 'feature_types'

# Downloading files

You can use [gsutil](https://cloud.google.com/storage/docs/gsutil) to download any of the files in the bucket
and work with them locally. 

Please be considerate to the [cost of egress](https://cloud.google.com/storage/pricing) when download the data from Google Cloud Storage.

For example:

```bash
gsutil cp gs://arc-ctc-scbasecamp/2025-02-25/h5ad/Homo_sapiens/ERX4319106.h5ad.gz .
```

For large data transfers, it is better to use `gsutil rsync`:

```bash
gsutil rsync gs://arc-ctc-scbasecamp/2025-02-25/h5ad/Callithrix_jacchus/ .
```

***

# sessionInfo

In [29]:
sessionInfo()

R version 4.2.3 (2023-03-15)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 22.04.3 LTS

Matrix products: default
BLAS/LAPACK: /home/nickyoungblut/miniforge3/envs/scbasecamp_renv/lib/libopenblasp-r0.3.29.so

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] googleCloudStorageR_0.7.0 reticulate_1.38.0        
[3] arrow_16.1.0              dplyr_1.1.4              

loaded via a namespace (and not attached):
 [1] zip_2.3.1         Rcpp_1.0.12       googleAuthR_2.0.2 pillar_1.9.0     
 [5] compiler_4.2.3    base64enc_0.1-3   tools_4.2.3       digest_0.6.36    
 [9] uuid_1.2-0        bit_4.0.5    